# PROJECT-4
## Решение комплексной бизнес-задачи

In [ ]:
# технический блок

import pandas as pd
import psycopg2
import itertools
import numpy as np
from collections import Counter

# Функция, которая вытаскивает данные из MetaBase по запросу. В нее необходимо передать код запроса на SQL.
def get_sql_data(query):
    try:
        connection = psycopg2.connect(
            dbname = 'skillfactory',
            user = 'skillfactory',
            host = '84.201.134.129',
            password = 'cCkxxLVrDE8EbvjueeMedPKt',
            port = 5432
        )
        curs = connection.cursor()
        curs.execute(query)
        data = curs.fetchall()
        connection.commit()
    except(Exception, psycopg2.Error) as error:
        if connection:
            print('Error', error)
    finally:
        if connection:
            curs.close()
    connection.close()
    return data

# Функция, возвращающая рекомендованные курсы в соответствии с уже выбранным. В нее необходимо передать
# идентификатор выбранного курса, к которому нужны рекомендации, топ популярных пар и топ популяроности курсов,
# для подбора рекомендаций в случаях, когда нельзя подобрать популярную пару.
def rec(resource_id):
    rec_pair = []
    for pair in selection_top:
        if resource_id in pair:
            rec_pair.append(pair[pair.index(resource_id)-1])
        if len(rec_pair)==2: break
    
    if len(rec_pair)==0:
        rec_pair.extend(top_courses[0:2])
        
    if len(rec_pair)==1:
        rec_pair.append(top_courses[0])
        
    return rec_pair
        
    

### Знакомство с данными

Определим за какие годы у нас есть данные о продажах:

In [ ]:
query = '''SELECT extract(year FROM purchased_at) AS purch_year
FROM final.carts
GROUP BY purch_year
'''
display(get_sql_data(query))

Определим количество клиентов, которые покупали курсы за рассматриваемый период:

In [ ]:
query = '''SELECT count(DISTINCT user_id)
FROM final.cart_items AS i
INNER JOIN final.carts AS c ON c.ID = i.cart_id
WHERE resource_type LIKE 'Course'
	AND STATE LIKE 'successful'
'''
display(get_sql_data(query))

Определим количество уникальных курсов в базе и их популярность:

In [ ]:
query = '''SELECT resource_id
FROM final.cart_items AS i
INNER JOIN final.carts AS c ON c.ID = i.cart_id
WHERE resource_type LIKE 'Course'
'''

courses_data = get_sql_data(query)

all_courses = list()

for course in courses_data:
    all_courses.append(course[0])

# Рассмотрим уникальные курсы:
unique_courses = list(set(all_courses))
print('В базе найдено', len(unique_courses), 'уникальных курсов')

# Выделим наиболее популярные курсы, которые будут учтены в рекомендациях:
top_courses = list(pd.Series(all_courses).value_counts().sort_values(ascending=False).keys()[0:2])
print('Два наиболее популярных курса:', top_courses)

Подсчитаем среднее число купленных курсов на одного клиента:

In [ ]:
query = '''SELECT avg(q.quantity)
FROM (
	SELECT count(DISTINCT resource_id) AS quantity
	FROM final.cart_items AS i
	INNER JOIN final.carts AS c ON c.ID = i.cart_id
	WHERE resource_type LIKE 'Course'
		AND STATE LIKE 'successful'
	GROUP BY user_id
	) AS q
 '''
display(get_sql_data(query))

Определим сколько клиентов купили больше одного курса:

In [ ]:
query = '''SELECT count(q.user_id)
FROM (
	SELECT user_id, count(DISTINCT resource_id) AS quantity
	FROM final.cart_items AS i
	INNER JOIN final.carts AS c ON c.ID = i.cart_id
	WHERE resource_type LIKE 'Course'
		AND STATE LIKE 'successful'
	GROUP BY user_id
	) AS q
WHERE quantity > 1
'''
display(get_sql_data(query))

### Подготовка данных

Подготовим данные по продажам курсов в разрезе пользователей и разобьем все покупки курсов по парам:

In [ ]:
query = '''WITH bulk_buyers
AS (
	SELECT user_id
	FROM final.cart_items AS i
	INNER JOIN final.carts AS c ON c.ID = i.cart_id
	WHERE resource_type LIKE 'Course'
		AND STATE LIKE 'successful'
	GROUP BY user_id
	HAVING count(DISTINCT resource_id) > 1
	ORDER BY 1
	)
SELECT DISTINCT user_id, resource_id
FROM final.cart_items AS i
INNER JOIN final.carts AS c ON c.ID = i.cart_id
WHERE resource_type LIKE 'Course'
	AND STATE LIKE 'successful'
	AND user_id IN (
		SELECT *
		FROM bulk_buyers
		)
ORDER BY 1,2
'''

data = get_sql_data(query)

# Преобразуем полученные данные в DataFrame:
df = pd.DataFrame(data=data, columns = ['user_id', 'resource_id'])
df_courses = df.groupby(by='user_id')['resource_id'].agg([list])

# Сформируем список пар курсов:
courses_pairs = list()
for courses in df_courses['list']:
    for courses_pair in itertools.combinations(courses, 2):
        courses_pairs.append(courses_pair)

# Преобразуем наш список в Series и воспользуемся агригацией для определения количества уникальных пар:
pairs_series = pd.Series(courses_pairs).sort_values()
print('В базе найдено', pairs_series.nunique(), 'уникальных пар курсов, которые встречаются вместе в покупках клиентов')

# Определим уровень популярность пар курсов:
rating_pairs = Counter(courses_pairs).most_common()
print('Самая популярная пара курсов:', rating_pairs[0][0])

### Формирование итогового результата

Сформируем таблицу, которая станет основой рекомендательной системы:

In [ ]:
# Опеределим нижнюю границу, как квантиль 25%, так как выбросов в данных не замечено:
rating_value = []
for i in rating_pairs:
    rating_value.append(i[1])
lower_limit = np.quantile(rating_value, 0.25)

# Сформируем упорядоченную выборку пар для подбора рекомендаций
selection_top = []
for i in rating_pairs:
    if i[1]>lower_limit:
        selection_top.append(i[0])


# Сформируем таблицу, в которой:
# resource_id - идентификатор курса, к которому подобраны рекомендации
# rec1 - отдельно иднтификатор первого рекомендованного курса
# rec2 - отдельно идентификатор второго рекомендованного курса.
df = pd.DataFrame({'resource_id': unique_courses})
df['rec'] = df['resource_id'].apply(rec)
df['rec1'] = df['rec'].apply(lambda x: x[0])
df['rec2'] = df['rec'].apply(lambda x: x[1])
df = df.drop('rec', axis=1)

display(df)
